In [1]:
import argparse
import random

import numpy as np
import torch

args = {
    "seed": 1234,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

args = argparse.Namespace(**args)
print(args)

Namespace(seed=1234, device=device(type='cuda'))


In [2]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

In [3]:
raw_inputs = [
    "나는 학생 입니다",
    "나는 좋은 선생님 입니다",
    "당신은 매우 좋은 선생님 입니다"
]

# 학생 : 1, 기타 : 0
raw_labels = [1, 0, 0]

words = []
for s in raw_inputs:
    words.extend(s.split())

words = list(dict.fromkeys(words))
word_to_id = {"[PAD]": 0, "[UNK]": 1}
for w in words:
    word_to_id[w] = len(word_to_id)

print(word_to_id)

id_to_word = {i: w for w, i in word_to_id.items()}
print(id_to_word)

{'[PAD]': 0, '[UNK]': 1, '나는': 2, '학생': 3, '입니다': 4, '좋은': 5, '선생님': 6, '당신은': 7, '매우': 8}
{0: '[PAD]', 1: '[UNK]', 2: '나는', 3: '학생', 4: '입니다', 5: '좋은', 6: '선생님', 7: '당신은', 8: '매우'}


In [12]:
onehot_metrix = np.eye(len(word_to_id))
print(onehot_metrix)

train_inputs = []
for s in raw_inputs:
    row = [word_to_id[w] for w in s.split()]
    row += [0] * (5-len(row)) # PAD
    train_inputs.append(row)
train_inputs = np.array(train_inputs)

print(train_inputs)

train_onehot = onehot_metrix[train_inputs]
print(train_onehot)

a = np.argmax(train_onehot, axis=-1)
print(np.argmax(train_onehot, axis=-1))

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[2 3 4 0 0]
 [2 5 6 4 0]
 [7 8 5 6 4]]
[[[0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0.]]]
[[2 3 4 0 0]
 [2 5 6 4 0]
 [7 8 5 6 4]]


In [13]:
t_train_inputs = torch.tensor(train_inputs)
t_train_onehots = torch.tensor(train_onehot).to(torch.float32)

embed = torch.nn.Embedding(len(word_to_id), 4)

hidden1 = embed(t_train_inputs)
print(hidden1)

print(embed.weight)

tensor([[[-0.2153,  0.8840, -0.7584, -0.3689],
         [-0.3424, -1.4020,  0.3206, -1.0219],
         [ 0.7988, -0.0923, -0.7049, -1.6024],
         [-0.1117, -0.4966,  0.1631, -0.8817],
         [-0.1117, -0.4966,  0.1631, -0.8817]],

        [[-0.2153,  0.8840, -0.7584, -0.3689],
         [-0.5675, -0.2772, -2.1834,  0.3668],
         [ 0.7667,  0.0190,  0.0220,  1.1532],
         [ 0.7988, -0.0923, -0.7049, -1.6024],
         [-0.1117, -0.4966,  0.1631, -0.8817]],

        [[ 1.8409, -1.0174,  1.2192,  0.1601],
         [-0.6857, -0.0496, -1.2485, -0.8509],
         [-0.5675, -0.2772, -2.1834,  0.3668],
         [ 0.7667,  0.0190,  0.0220,  1.1532],
         [ 0.7988, -0.0923, -0.7049, -1.6024]]], grad_fn=<EmbeddingBackward0>)
Parameter containing:
tensor([[-0.1117, -0.4966,  0.1631, -0.8817],
        [ 0.0539,  0.6684, -0.0597, -0.4675],
        [-0.2153,  0.8840, -0.7584, -0.3689],
        [-0.3424, -1.4020,  0.3206, -1.0219],
        [ 0.7988, -0.0923, -0.7049, -1.6024],
       